In [1]:
from artifacts_ipv6_sra_scanning.config import *
import pickle
import os
import ipaddress
import glob
from tqdm.auto import tqdm
import country_converter as coco
from parallel_pandas import ParallelPandas
ParallelPandas.initialize(n_cpu=24, split_factor=1, disable_pr_bar=False)

2025-10-06 12:39:19.464 | INFO     | artifacts_ipv6_sra_scanning.config:<module>:22 - PROJ_ROOT path is: /home/service/artifacts-ipv6-sra-scanning


In [2]:
df_tum_sra_as = pl.read_csv(f'{INTERIM_DATA_DIR}/init-ips-sra-tum-scan.csv')

In [3]:
df_tum_reply_as = pl.read_parquet(f'{PROCESSED_DATA_DIR}/sra_tum_scans_stability.parquet')

In [6]:
df_router_ips = pl.read_csv(f'{INTERIM_DATA_DIR}/router-ips.csv')

In [7]:
df_router_ips.head()

ip-addr,Geo,AS-Number,BGP-Prefix,Org
str,str,f64,str,str
"""2402:b400:4440:53ef:c822:8bff:…","""MYS""",45960.0,"""2402:b400:4440::/46""","""YTLCOMMS-AS-AP YTL COMMUNICATI…"
"""2003:e6:37ff:3ab2:3ea6:2fff:fe…","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…"
"""2405:8d40:4081:579:1807:c46a:e…","""PHL""",139831.0,"""2405:8d40:4080::/42""","""DTC-AS-AP DITO TELECOMMUNITY C…"
"""240e:358:9c0a:13ec:ae64:62ff:f…","""CHN""",4134.0,"""240e:358::/29""","""CHINANET-BACKBONE No.31,Jin-ro…"
"""2408:8441:5808:10fa:1807:f6b5:…","""CHN""",4837.0,"""2408:8441::/32""","""CHINA169-BACKBONE CHINA UNICOM…"


In [8]:
df_tum_sra_as

initial-ip,Geo,AS-Number,BGP-Prefix,Org
str,str,f64,str,str
"""240e:358:e00:6b54::""","""CHN""",4134.0,"""240e:358::/29""","""CHINANET-BACKBONE No.31,Jin-ro…"
"""2409:40e4:8:7db2::""","""IND""",55836.0,"""2409:40e4::/36""","""RELIANCEJIO-IN Reliance Jio In…"
"""2001:1670:10:6362::""","""OMN""",28885.0,"""2001:1670:10::/45""","""OMANTEL-NAP-AS OmanTel NAP"""
"""2003:e3:3719:2e00::""","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…"
"""2400:2412:4761:b600::""","""JPN""",17676.0,"""2400:2000::/20""","""GIGAINFRA SoftBank Corp."""
…,…,…,…,…
"""2003:c2:47ff:2156::""","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…"
"""2804:1e68:c201:23d0::""","""BRA""",53006.0,"""2804:1e68::/32""","""ALGAR TELECOM SA"""
"""2a01:e0a:31:eec0::""","""FRA""",12322.0,"""2a01:e0a::/39""","""PROXAD"""


In [5]:
df_tum_reply_as.head()

initial-ip,saddr,classification,saddr_s1,classification_s1,saddr_s2,classification_s2,saddr_s3,classification_s3,saddr_s4,classification_s4,saddr_s5,classification_s5
str,str,str,str,str,str,str,str,str,str,str,str,str
"""240e:358:e00:6b54::""","""240e:358:e00:6b54:7869:e171:7f…","""echoreply""","""240e:358:e00:6b54:7869:e171:7f…","""echoreply""","""240e:358:e00:6b54:7869:e171:7f…","""echoreply""",null,null,null,null,null,null
"""2409:40e4:8:7db2::""","""2409:40e4:8:7db2:5e2f:65c5:7b5…","""unreach_noroute""","""2409:40e4:8:7db2:5e2f:65c5:7b5…","""unreach_noroute""","""2409:40e4:8:7db2:5e2f:65c5:7b5…","""unreach_noroute""",null,null,null,null,null,null
"""2001:1670:10:6362::""","""2001:1670:10:6362:1008:d0df:88…","""echoreply""","""2001:1670:10:6362:1008:d0df:88…","""echoreply""","""2001:1670:10:6362:1008:d0df:88…","""echoreply""","""2001:1670:14:0:40a4:5e47:67f0:…","""unreach_addr""","""2001:1670:10:6362:4dbb:9498:1d…","""echoreply""","""2001:1670:10:6362:4dbb:9498:1d…","""echoreply"""
"""2003:e3:3719:2e00::""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin""","""2003:e3:37ff:19a5:e228:6dff:fe…","""unreach_admin"""
"""2400:2412:4761:b600::""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply""","""2400:2412:4761:b601:3ae0:8eff:…","""echoreply"""


In [10]:
tmp = df_tum_sra_as.join(df_tum_reply_as.select(['initial-ip','saddr']),how='left',on='initial-ip')

In [11]:
tmp

initial-ip,Geo,AS-Number,BGP-Prefix,Org,saddr
str,str,f64,str,str,str
"""240e:358:e00:6b54::""","""CHN""",4134.0,"""240e:358::/29""","""CHINANET-BACKBONE No.31,Jin-ro…","""240e:358:e00:6b54:7869:e171:7f…"
"""2409:40e4:8:7db2::""","""IND""",55836.0,"""2409:40e4::/36""","""RELIANCEJIO-IN Reliance Jio In…","""2409:40e4:8:7db2:5e2f:65c5:7b5…"
"""2001:1670:10:6362::""","""OMN""",28885.0,"""2001:1670:10::/45""","""OMANTEL-NAP-AS OmanTel NAP""","""2001:1670:10:6362:1008:d0df:88…"
"""2003:e3:3719:2e00::""","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…","""2003:e3:37ff:19a5:e228:6dff:fe…"
"""2400:2412:4761:b600::""","""JPN""",17676.0,"""2400:2000::/20""","""GIGAINFRA SoftBank Corp.""","""2400:2412:4761:b601:3ae0:8eff:…"
…,…,…,…,…,…
"""2003:c2:47ff:2156::""","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…","""2003:c2:47ff:2156:ca0e:14ff:fe…"
"""2804:1e68:c201:23d0::""","""BRA""",53006.0,"""2804:1e68::/32""","""ALGAR TELECOM SA""","""2804:1e68:c200:968:e549:1ba1:d…"
"""2a01:e0a:31:eec0::""","""FRA""",12322.0,"""2a01:e0a::/39""","""PROXAD""","""2a01:e0a:31:eec0::1"""


In [12]:
tmp = tmp.join(df_router_ips,how='left',left_on='saddr',right_on='ip-addr',suffix='_saddr')

In [13]:
tmp

initial-ip,Geo,AS-Number,BGP-Prefix,Org,saddr,Geo_saddr,AS-Number_saddr,BGP-Prefix_saddr,Org_saddr
str,str,f64,str,str,str,str,f64,str,str
"""240e:358:e00:6b54::""","""CHN""",4134.0,"""240e:358::/29""","""CHINANET-BACKBONE No.31,Jin-ro…","""240e:358:e00:6b54:7869:e171:7f…","""CHN""",4134.0,"""240e:358::/29""","""CHINANET-BACKBONE No.31,Jin-ro…"
"""2409:40e4:8:7db2::""","""IND""",55836.0,"""2409:40e4::/36""","""RELIANCEJIO-IN Reliance Jio In…","""2409:40e4:8:7db2:5e2f:65c5:7b5…","""IND""",55836.0,"""2409:40e4::/36""","""RELIANCEJIO-IN Reliance Jio In…"
"""2001:1670:10:6362::""","""OMN""",28885.0,"""2001:1670:10::/45""","""OMANTEL-NAP-AS OmanTel NAP""","""2001:1670:10:6362:1008:d0df:88…","""OMN""",28885.0,"""2001:1670:10::/45""","""OMANTEL-NAP-AS OmanTel NAP"""
"""2003:e3:3719:2e00::""","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…","""2003:e3:37ff:19a5:e228:6dff:fe…","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…"
"""2400:2412:4761:b600::""","""JPN""",17676.0,"""2400:2000::/20""","""GIGAINFRA SoftBank Corp.""","""2400:2412:4761:b601:3ae0:8eff:…","""JPN""",17676.0,"""2400:2000::/20""","""GIGAINFRA SoftBank Corp."""
…,…,…,…,…,…,…,…,…,…
"""2003:c2:47ff:2156::""","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…","""2003:c2:47ff:2156:ca0e:14ff:fe…","""DEU""",3320.0,"""2003::/19""","""DTAG Internet service provider…"
"""2804:1e68:c201:23d0::""","""BRA""",53006.0,"""2804:1e68::/32""","""ALGAR TELECOM SA""","""2804:1e68:c200:968:e549:1ba1:d…","""BRA""",53006.0,"""2804:1e68::/32""","""ALGAR TELECOM SA"""
"""2a01:e0a:31:eec0::""","""FRA""",12322.0,"""2a01:e0a::/39""","""PROXAD""","""2a01:e0a:31:eec0::1""","""FRA""",12322.0,"""2a01:e0a::/39""","""PROXAD"""


In [15]:
tmp.select('AS-Number').n_unique()

18008

In [16]:
tmp.select('AS-Number_saddr').n_unique()

13994

In [17]:
tmp.filter(pl.col('AS-Number')!=pl.col('AS-Number_saddr'))

initial-ip,Geo,AS-Number,BGP-Prefix,Org,saddr,Geo_saddr,AS-Number_saddr,BGP-Prefix_saddr,Org_saddr
str,str,f64,str,str,str,str,f64,str,str
"""2a00:5400:f002:72b1::""","""SAU""",35753.0,"""2a00:5400::/32""","""ITC ITC AS number""","""2a00:5400:f002:72b1:694:b204:e…","""SAU""",43766.0,"""2a00:5400:f002::/48""","""MTC-KSA-AS"""
"""2600:100c:a122:824f::""","""USA""",6167.0,"""2600:100c:a122::/48""","""CELLCO-PART""","""2600:100c:a122:824f:15be:d6e7:…","""USA""",22394.0,"""2600:100c::/32""","""CELLCO"""
"""2800:484:8b74:4300::""","""COL""",10620.0,"""2800:484:8b00::/40""","""Telmex Colombia S.A.""","""2800:485:0:6d:5559:8e90:4ace:4…","""COL""",14080.0,"""2800:485::/32""","""Telmex Colombia S.A."""
"""2800:484:b16f:9f00::""","""COL""",10620.0,"""2800:484:b100::/40""","""Telmex Colombia S.A.""","""2800:485:0:b6:e0fc:591c:ef1c:b…","""COL""",14080.0,"""2800:485::/32""","""Telmex Colombia S.A."""
"""2800:484:9085:7900::""","""COL""",10620.0,"""2800:484:9000::/40""","""Telmex Colombia S.A.""","""2800:485:0:7b:586f:c8ac:e507:6…","""COL""",14080.0,"""2800:485::/32""","""Telmex Colombia S.A."""
…,…,…,…,…,…,…,…,…,…
"""2001:1a10:1120:400::""","""QAT""",42298.0,"""2001:1a10:1120::/44""","""GCC-MPLS-PEERING GCC MPLS peer…","""2001:1a10:a00:2:4cca:5880:1204…","""QAT""",8781.0,"""2001:1a10::/32""","""QA-ISP"""
"""2803:9800:a012:8259::""","""ARG""",11664.0,"""2803:9800:a012::/48""","""Techtel LMDS Comunicaciones In…","""2800:190:4001:11:456c:716c:a22…","""ARG""",19037.0,"""2800:190::/32""","""AMX Argentina S.A."""
"""240e:47e:30f8:3c58::""","""CHN""",134774.0,"""240e:47e:30f8::/45""","""CHINANET-GUANGDONG-SHENZHEN-MA…","""240e:47e:30f8:3c58:c413:98e0:f…","""CHN""",4134.0,"""240e:47c::/30""","""CHINANET-BACKBONE No.31,Jin-ro…"


In [18]:
tmp.write_parquet(f'{PROCESSED_DATA_DIR}/tum-sra-AS-comparison.parquet')